# 01 - Python Exceptions

When an exception is raised, Python triggers a special execution **propagation** workflow.

If the current call does not handle the exception, it is propagated up the stack to the caller. If the exception isn't handled anywhere throughout our code, then it propagates to the module level which terminates our program.

In-built Python exceptions are arranged in the following hierarchy:
- `BaseException`
    - `SystemExit` (raised on `sys.exit()`)
    - `KeyboardInterrupt` (raised on Ctrl+C for example)
    - `GeneratorExit` (raised when generator or coroutine is closed)
    - `Exception` -> Everything else. Subclasses of this includes `ValueError`, `TypeError`, `SyntaxError`, `RuntimeError`

The structure of total exception handling is the following:
```python
try:
    <code we want to protect from exceptions>
    
except <ExceptionType> as ex:
    <code that runs if specified <ExceptionType> occurs (or any subclass)> # This should be the most specific exception.

except <ExceptionType2> as ex2:
    <code that runs if specified <ExceptionType> occurs (or any subclass)> # This should be less specific than the first exception.
    
finally:
    <code that always executes whether exception occured or not>
    
else:
    <code that executes if try terminates without exceptions>
```
The `as` keyword in the `except` block gives us a handle to the **instance** of the exception.

Here is a basic example to highlight the importance of avoiding bare exceptions. We are iterating through an iterable and squaring each value. Ideally we want to guard against calling `squares(seq, max_n)` with a `max_n` greater than the length of `seq`.

In [16]:
def square(seq, index):
    return seq[index] ** 2

def squares(seq, max_n):
    for i in range(max_n):
        try:
            yield square(seq, i)
        except:
            return

In [17]:
l = [1, 2, 3, 4, 5]
list(squares(l, 10))

[1, 4, 9, 16, 25]

This ran successfully as expected as we caught the `IndexError` due to `max_n` being greater than the length of the sequence. 

In [18]:
l = [1, 2, '3', 4, 5]
list(squares(l, 10))

[1, 4]

This should've terminated with a meaningful output as `seq['3'] ** 2` should raise a **TypeError**. But it actually ran successfully.

This was because this exception was caught by our bare exception causing our program to terminate, but really, we would want this error to bubble up as we only intended to guard against `IndexErrors`. Fixing this, we get the desired bubble-up: 

In [12]:
def square(seq, index):
    return seq[index] ** 2

def squares(seq, max_n):
    for i in range(max_n):
        try:
            yield square(seq, i)
        except IndexError:
            return

In [13]:
l = [1, 2, '3', 4, 5]
list(squares(l, 10))

TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

# 02 - Handling Exceptions

#### Lecture

One important rule of them regarding exception handling is to **only guard code where you can do something about the exception**.

If you can't do anything about it, then terminating your code is a valid option

As seen in the previous subsection, we can have multiple exceptions for a given `try`. 

We should ensure that our `except` blocks go from most specific to least (in terms of class hierarchy) as the first `except` statement that matches will run which will prevent any further `excepts` from running.

We can also handle **multiple** exceptions in the **same** way by grouping them into a tuple:
```python
try:
    ...
except (ValueError, TypeError) as ex:
    log(ex)
```

Regarding **Bare Exceptions** (`except:`), although these are discouraged, they can be useful if we want to log all exceptions before re-raising the same exception.

In this scenario, we may want a handle on the exception object. We can do this by calling `sys.exc_info()` within the exception clause which returns the following tuple: `(exc_type, exc_value, exc_traceback)`.

**Exception Objects**

The properties and methods of an exception depend on the exception. Standard exceptions always have at least these two properties:
- `args`: arguments used to create exception object - often just the error message we want to be printed out - used in `str` and `repr` methods.
- `__traceback__`: the traceback object. Rarely used in practice but can be useful for logging.

**Handling Exceptions vs Avoiding Exceptions**

"It's easier to ask for forgiveness than it is to get permission" (EAFP). 

In Python, we tend to prefer this principle which is facilitated with `try-except` as opposed to convoluted, nested `if-else` blocks.

This is because the latter approach makes it difficult to determine all possible things that can go wrong, such as forgetting to close files, or all possibilities to consider. 

For example, how do we conclusively determine if something is a sequence? We could use `hasattr(<seq>, '__getitem__')` but dictionaries implement this despite not being traditional sequences. The simplest way is just to `try: seq[idx]` and catch any exceptions.

#### Code

Printing the exception object utilises the string representation which prints our custom message. If we provided multiple arguments, these appear comma-separated like a tuple, but it's a *string*.

In [6]:
try:
    raise ValueError('custom message', 'secondary message')
except ValueError as ex:
    print(ex)

('custom message', 'secondary message')


We can get each individual argument back using the `args` property:

In [35]:
try:
    raise ValueError('custom message', 'secondary message')
except ValueError as ex:
    print(ex.args[1])

secondary message


For extra information, we can use `repr`:

In [2]:
try:
    raise ValueError('custom message', 'secondary message')
except ValueError as ex:
    print(repr(ex))

ValueError('custom message', 'secondary message')


Consider the following example:

In [8]:
try:
    a = 10
except ValueError:
    print('value error...')
else:
    print('no exception occurred...')

no exception occurred...


Some developers are tempted to ignore the `else` clause altogether, and write the following:

In [9]:
try:
    a = 10
except ValueError:
    print('value error...')

print('no exception occurred...')

no exception occurred...


Although this *may* seem identical and intuitive as `else` clauses in `if` blocks can be omitted without loss of functionality, this is **not** the case here.

If an excpeption *is* caught, the difference between these two examples is demonstrated:

In [10]:
try:
    raise ValueError()
except ValueError:
    print('value error...')
else:
    print('no exception occurred...')

value error...


In [11]:
try:
    raise ValueError()
except ValueError:
    print('value error...')

print('no exception occurred...')

value error...
no exception occurred...


#### Example

Here we want to create a simple function to transform `0`, `1`, `"0"`, `"1"`, `"T"`, `"F"`, `"True"`, `"False"`, `True` and `False` into the equivalent boolean type, as well as case insensitive versions of the strings.

In [12]:
class ConversionError(Exception):
    pass

In [13]:
def convert_int(val):
    if not isinstance(val, int):  # remember this will work for booleans too!
        raise TypeError()
    if val not in {0, 1}:
        raise ValueError("Integer values 0 or 1 only")
    return bool(val)

In [14]:
def convert_str(val):
    if not isinstance(val, str):
        raise TypeError()
        
    val = val.casefold()  # for case-insensitive comparisons
    if val in {'0', 'f', 'false'}:
        return False
    elif val in {'1', 't', 'true'}:
        return True
    else:
        raise ValueError('Admissible string values are: T, F, True, False (case insensitive)')

In [15]:
def make_bool(val):
    for converter in (convert_int, convert_str):
        try:
            return converter(val)
        except TypeError:
            error_msg = f'The type {type(val).__name__} cannot be converted to a bool'
        except ValueError as ex:
            error_msg = f'The value {val} cannot be converted to a bool: {ex}'
            break
 
    raise ConversionError(error_msg)

In [16]:
values = [True, 0, 'T', 'false', 10, 'ABC', 1.0]

for value in values:
    try:
        result = make_bool(value)
    except ConversionError as ex:
        result = str(ex)

    print(value, result)

True True
0 False
T True
false False
10 The value 10 cannot be converted to a bool: Integer values 0 or 1 only
ABC The value ABC cannot be converted to a bool: Admissible string values are: T, F, True, False (case insensitive)
1.0 The type float cannot be converted to a bool


# 03 - Raising Exceptions

Raising exceptions are simple. We just need to ensure that we raise an object that inherits from `BaseException`, not necessarily directly.

If we're within an `except` clause and we want to perform some cleanup, logging etc., before **re-raising** our current exception, all we need to do is write `raise` without specifying an object. This will resume the exception propagation as before. Here's an example with a perfectly acceptable **bare** exception:
```python
try:
    ...
except:
    log(...)
    raise
```

**Limiting Traceback Output with `from`**

Sometimes, we may only want to print the final exception in a deeply nested exception block. We might want to do this to avoid overwhelming the user or to hide internal implementation details.

In [13]:
try:
    raise ValueError('level 1')
except ValueError:
    try:
        raise TypeError('level 2')
    except TypeError:
        raise KeyError('level 3')

KeyError: 'level 3'

We do this with the following syntax: `Raise <ExceptionType> from None`:

In [26]:
try:
    raise ValueError('level 1')
except ValueError:
    try:
        raise ValueError('level 2')
    except ValueError:
        try:
            raise ValueError('level 3') 
        except:
            raise ValueError('Cannot recover') from None

ValueError: Cannot recover

We can also bypass exceptions by passing in the `exception` object whose traceback we want to use. In the example above, let's say we only want to display the traceback associated with the first `try` if we reach the final 'Cannot recover' exception. Here's how we do it:

In [30]:
try:
    raise ValueError('level 1')
except ValueError as ex_1:  # specify the handle of the tb object
    try:
        raise ValueError('level 2')
    except ValueError:
        try:
            raise ValueError('level 3') 
        except:
            raise ValueError('Cannot recover') from ex_1  # utilise handle

ValueError: Cannot recover

# 04 - Custom Exceptions

#### Lecture

For the most simple custom exception cases, it's sufficient to create a class that inherits from `Exception` with a docstring:

In [31]:
class WidgetException(Exception):
    """base custom exception for the widget library"""

But we can add additional functionality and even override special methods such as `str` and `repr`.

For example, we can add **auto-logging** to your exceptions, or create a suitable **json** representation for our exception which can be passed around with a consistent structure.

**Multiple Inheritance**

Choosing the appropriate exception type to catch can be challenging if multiple seem applicable. 

So, sometimes it might make sense to create a custom exception that inherits from **all**. This custom exception will be raised if **any** of the multiple exceptions are raised.

In [34]:
class SalesException(Exception):
    """base custom exception for the Sales module"""

class InvalidSalePrice(SalesException, ValueError):
    """raised if either exception is seen"""

try:
    raise ValueError('Invalid Value')
except InvalidSalePrice:
    print(ex)

ValueError: Invalid Value

#### Example

Often when we have a relatively complex application, we create our own hierarchy of exceptions, where we use some base exception for our application, and every other exception is a subclass of that exception.

For example, suppose we are writing a REST API. When we raise a custom exception, we'll also want to return an HTTP exception response to the API caller. We could write code like this in our API calls:

Suppose we need to retrieve an account (by ID) from a database. Here I'm just going to mock this:
```python
class APIException(Exception):
    """Base API exception"""
    
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""
    
class DBException(ApplicationException):
    """General database exception"""
    
class DBConnectionError(DBException):
    """Indicates an error connecting to database"""
    
class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""
    
class NotFoundError(ClientException):
    """Indicates resource was not found"""

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""
```

So we have this exception hierarchy:

```
APIException
   - ApplicationException (5xx errors)
       - DBException
           - DBConnectionError
   - ClientException
       - NotFoundError
       - NotAuthorizedError
```

The current limitation of this approach is we'll need to articulate what type of HTTP response should be returned for each exception after catching them. 

It would be much cleaner to articulate this within each exception class. For example, `NotFoundError` should be associated with the **HTTP 404** error.

Some more nice-to-haves:
- Bound to the base exception class should be a `to_json` method which bundles the exception information into a dictionary according to an appropriate schema.
- Bound to the base exception class should be a `log_exception` method which constructs a json string with the relevant exception information and (for now,) prints it out.

Here's our final base class:

In [14]:
from datetime import datetime
from http import HTTPStatus
import json

class APIException(Exception):
    """Base API exception"""
    
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = 'API exception occurred.'
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
    def __init__(self, *args, user_err_msg = None):
        if args:
            self.internal_err_msg = args[0]
            super().__init__(*args)
        else:
            super().__init__(self.internal_err_msg)
            
        if user_err_msg is not None:
            self.user_err_msg = user_err_msg
    
    def to_json(self):
        err_object = {'status': self.http_status, 'message': self.user_err_msg}
        return json.dumps(err_object)
    
    def log_exception(self):
        exception = {
            "type": type(self).__name__,
            "http_status": self.http_status,
            "message": self.args[0] if self.args else self.internal_err_msg,
            "args": self.args[1:]
        }
        print(f'EXCEPTION: {datetime.utcnow().isoformat()}: {exception}')

Here's our hierarchy where we override the class attributes in the base class.

In [15]:
class ApplicationException(APIException):
    """Indicates an application error (not user caused) - 5xx HTTP type errors"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Generic server side exception."
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
class DBException(ApplicationException):
    """General database exception"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "Database exception."
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
class DBConnectionError(DBException):
    """Indicates an error connecting to database"""
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    internal_err_msg = "DB connection error."
    user_err_msg = "We are sorry. An unexpected error occurred on our end."
    
class ClientException(APIException):
    """Indicates exception that was caused by user, not an internal error"""
    http_status = HTTPStatus.BAD_REQUEST
    internal_err_msg = "Client submitted bad request."
    user_err_msg = "A bad request was received."
    
class NotFoundError(ClientException):
    """Indicates resource was not found"""
    http_status = HTTPStatus.NOT_FOUND
    internal_err_msg = "Resource was not found."
    user_err_msg = "Requested resource was not found."

class NotAuthorizedError(ClientException):
    """User is not authorized to perform requested action on resource"""
    http_status = HTTPStatus.UNAUTHORIZED
    internal_err_msg = "Client not authorized to perform operation."
    user_err_msg = "You are not authorized to perform this request."

To test our code for now, we can mock the different exceptions in the following way.

Also, notice how we can pass extra information to our exception class, such as `"db=db01"`, which will get stored under `self.args` of our exception instance, and is subsequently passed to our logger.

In [25]:
class Account:
    def __init__(self, account_id, account_type):
        self.account_id = account_id
        self.account_type = account_type

In [26]:
def lookup_account_by_id(account_id):
    # mock of various exceptions that could be raised getting an account from database
    if not isinstance(account_id, int) or account_id <= 0:
        raise ClientException(f'Account number {account_id} is invalid.', 
                              f'account_id = {account_id}',
                              'type error - account number not an integer')
        
    if account_id < 100:
        raise DBConnectionError('Permanent failure connecting to database.', 'db=db01')
    elif account_id < 200:
        raise NotAuthorizedError('User does not have permissions to read this account', f'account_id={account_id}')
    elif account_id < 300:
        raise NotFoundError(f'Account not found.', f'account_id={account_id}')
    else:
        return Account(account_id, 'Savings')

Now we can re-write our API endpoint and very easily handle those exceptions:

Notice how we only need to catch the base class `APIException` instead of trying to handle each exception separately e.g. `ClientException`, `DBConnectionError`, `NotFoundError`, etc.

In [27]:
def get_account(account_id):
    try:
        account = lookup_account_by_id(account_id)
    except APIException as ex:
        ex.log_exception()
        return ex.to_json()
    else:
        return HTTPStatus.OK, {"id": account.account_id, "type": account.account_type}

Playing around with it:

In [19]:
get_account('ABC')

EXCEPTION: 2024-10-25T10:31:36.849004: {'type': 'ClientException', 'http_status': <HTTPStatus.BAD_REQUEST: 400>, 'message': 'Account number ABC is invalid.', 'args': ('account_id = ABC', 'type error - account number not an integer')}


'{"status": 400, "message": "A bad request was received."}'

In [20]:
get_account(50)

EXCEPTION: 2024-10-25T10:31:37.828073: {'type': 'DBConnectionError', 'http_status': <HTTPStatus.INTERNAL_SERVER_ERROR: 500>, 'message': 'Permanent failure connecting to database.', 'args': ('db=db01',)}


'{"status": 500, "message": "We are sorry. An unexpected error occurred on our end."}'

In [21]:
get_account(150)

EXCEPTION: 2024-10-25T10:31:38.146063: {'type': 'NotAuthorizedError', 'http_status': <HTTPStatus.UNAUTHORIZED: 401>, 'message': 'User does not have permissions to read this account', 'args': ('account_id=150',)}


'{"status": 401, "message": "You are not authorized to perform this request."}'

In [22]:
get_account(250)

EXCEPTION: 2024-10-25T10:31:38.318985: {'type': 'NotFoundError', 'http_status': <HTTPStatus.NOT_FOUND: 404>, 'message': 'Account not found.', 'args': ('account_id=250',)}


'{"status": 404, "message": "Requested resource was not found."}'

In [23]:
get_account(350)

(<HTTPStatus.OK: 200>, {'id': 350, 'type': 'Savings'})